In [5]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [6]:
api_key = "eTxxZELrPee92VjW2INmUtkj1D4pC7QG"
model = "open-mistral-7b"

client = MistralClient(api_key=api_key)


In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [46]:
search_results=elastic_search(query)
len(build_prompt(query,search_results))

1499

In [9]:
def llm(prompt,model):
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
    )

    print(chat_response.choices[0].message.content)

In [10]:
import json

In [11]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [12]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
from elasticsearch import Elasticsearch

In [20]:
es_client=Elasticsearch('http://localhost:9200')

In [22]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="course-questions"

es_client.indices.create(index=index_name, body= index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [23]:
from tqdm.auto import tqdm

In [24]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

In [25]:
query="How do I execute a command in a running docker container?"

In [44]:
def elastic_search(query):
    search_query={
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response=es_client.search(index=index_name, body=search_query)

    result_docs=[]
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [45]:
elastic_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [34]:
def rag(query,model):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt,model)
    return answer

In [35]:
rag(query,model)

To execute a command in a running Docker container, follow these steps:

1. First, find the container ID by running `docker ps`.

2. Once you have the container ID, use the `docker exec` command to execute a command within the specific container. In this case, let's use `bash` to open an interactive shell within the container:

```
docker exec -it <container-id> bash
```

Now you can execute any command within the container. For example, if you wanted to run a command like `ls` to list the files in the container, you would simply type `ls` and press enter within the container shell.


In [1]:
pip install mistral-common

Note: you may need to restart the kernel to use updated packages.


In [37]:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

In [38]:
tokenizer_v3 = MistralTokenizer.from_model(model)

In [49]:
prompt=build_prompt(query,search_results)

In [50]:
tokenized = tokenizer_v3.encode_chat_completion(
    ChatCompletionRequest(
        model=model,
        messages=[
            UserMessage(content=prompt),
        ],
    )
)
tokens, text = tokenized.tokens, tokenized.text

In [51]:
len(tokens)

417